In [1]:
from nodag import nodag, weight_to_adjacency
import numpy as np
from synthetic_dataset import SyntheticDataset
from MEC import is_in_markov_equiv_class
from nodag import weight_to_adjacency

c:\Users\super\DAG


In [2]:
!pip install cdt
from cdt.metrics import SHD_CPDAG


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


In [3]:
!pip install rpy2
import rpy2.robjects as ro
print(ro.r('1+1'))


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


[1] 2



In [4]:
import os
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

print("R.home() =", ro.r('R.home()')[0])
print(".libPaths() =", list(ro.r('.libPaths()')))

# 看 rpy2 实际能否加载 pcalg
try:
    importr('pcalg')
    print('pcalg import OK')
except Exception as e:
    print('pcalg NOT found ->', e)

R.home() = D:/coding/R-4.5.1
.libPaths() = ['D:/coding/R-4.5.1/library']
pcalg import OK


In [5]:
from itertools import permutations
from numpy.linalg import inv, cholesky, LinAlgError


def find_L_with_best_l1(Sigma, verbose = False):
    d = len(Sigma[0])
    best_L, best_P = None, None
    best_l1 = np.inf
    for perm in permutations(range(d)):
        P = np.eye(d)[list(perm)]  # permutation matrix

        # Step 2: permute covariance
        Sigma_perm = P @ Sigma @ P.T

        try:
            # Step 3: compute precision matrix
            Theta = inv(Sigma_perm)
            # Step 4: Cholesky decomposition (Theta = L L^T)
            L = cholesky(Theta)

            l1_norm = np.linalg.norm(L, ord=1)
            
            # print("score =",score)
            if l1_norm < best_l1:
                best_l1 = l1_norm
                best_P = P
                best_L = P.T @ L @ P
                
                if verbose:
                    print(f"New best score {l1_norm} with permutation {perm}")
        except LinAlgError:
            continue

    return best_L, best_P, best_l1


In [6]:
def l0_norm(U, threshold=0.05):
    """
    Count the number of non-zero off-diagonal elements in the upper triangular matrix U.
    """
    p = U.shape[0]
    count = 0
    for i in range(p):
        for j in range(p):
            if abs(U[i, j]) > threshold:
                count += 1
    return count

In [7]:

from scipy.linalg import sqrtm

n, d = 10000, 8
graph_type, degree = 'ER', 2    # ER2 graph
B_scale = 1.0
noise_type = 'gaussian_nv'
times = 100
l0_tol_est = 0
l0_tol_true = 0
l1_tol_true = 0
l1_tol_est = 0
proportion = 0
shd_cpdag_tol = 0
for t in range(times):
    dataset = SyntheticDataset(n, d, graph_type, degree,
                            noise_type, B_scale, seed = t)
    X = dataset.X.T
    #print(X.shape)
    B_true = dataset.B_bin
    W_true = dataset.B
    N = dataset.N
    Omega_true = np.diag(N)
    Sigma_hat = np.cov(X)
    Theta_hat = inv(Sigma_hat)
    #print(Sigma_hat.shape)
    Theta_true = (np.eye(d) - W_true) @ inv(Omega_true) @ (np.eye(d) - W_true.T)
    Sigma_true = inv(Theta_true)

    L_true = (np.eye(d) - W_true) @ inv(sqrtm(Omega_true))
    likelihood_true = - 2 * np.log(np.linalg.det(L_true)) + np.trace(L_true.T @ Sigma_true @ L_true)

    likelihood_hat = - 2 * np.log(np.linalg.det(L_true)) + np.trace(L_true.T @ Sigma_hat @ L_true)
    l1_true = np.linalg.norm(L_true, ord=1)

    #print("likelihood_true = ", likelihood_true)
    #print("likelihood_hat = ", likelihood_hat)
    #print("l1_true = ", l1_true)

    L_est, P_est, l1_est = find_L_with_best_l1(Sigma_hat)
    W_est = np.eye(d) - L_est
    B_est = weight_to_adjacency(W_est, 0.1)
    likelihood_est = - 2 * np.log(np.linalg.det(L_est)) + np.trace(L_est.T @ Sigma_hat @ L_est)
    l1_est = np.linalg.norm(L_est, ord=1)

    l1_tol_true += l1_true
    l1_tol_est  += l1_est
    l0_tol_est += l0_norm(L_est)
    l0_tol_true += l0_norm(L_true)
    shd_cpdag_tol += SHD_CPDAG(B_true, B_est)

    if l1_est < l1_true:
        proportion += 1

    #print("likelihood_est = ", likelihood_est)
    #print("l1_est = ", l1_est)

    #print("L_true l0 =\n ", l0_norm(L_true))
    #print("L_est l0=\n ", l0_norm(L_est))

l1_est_avg = l1_tol_est / times
l1_true_avg = l1_tol_true / times
l0_est_avg = l0_tol_est / times
l0_true_avg = l0_tol_true / times
proportion /= 100
shd_cpdag_avg = shd_cpdag_tol / 100

print("Average true L1 = ",l1_true_avg)
print("Average estimated L1 = ", l1_est_avg)
print("Average true L0 = ",l0_true_avg)
print("Average estimated L0 = ", l0_est_avg)
print("Proportion of estimated L1 < true L1", proportion)
print("SHD_CPDAG = ", shd_cpdag_avg)



    

Average true L1 =  3.8768266368515487
Average estimated L1 =  3.014575555181141
Average true L0 =  16.09
Average estimated L0 =  19.72
Proportion of estimated L1 < true L1 0.91
SHD_CPDAG =  11.58


In [13]:
b1 = np.array([[0, 0.5, 0],[0, 0, 0],[0, -1, 0]])
b2 = np.array([[0, 0.5, 0.1],[0, 0, -0.2], [0, 0, 0]])
Omega1 = np.array([[16, 0, 0],[0, 4, 0],[0, 0, 1]])
Omega2 = np.array([[16, 0, 0],[0 ,5, 0],[0, 0, 0.8]])

l1 = (np.eye(3) - b1) @ inv(sqrtm(Omega1))
l2 = (np.eye(3) - b2) @ inv(sqrtm(Omega2))

Theta1 = (np.eye(3) - b1) @ inv(Omega1) @ (np.eye(3) - b1.T)
Theta2 = (np.eye(3) - b2) @ inv(Omega2) @ (np.eye(3) - b2.T)

Sigma1 = inv(Theta1)
Sigma2 = inv(Theta2)

print(- 2 * np.log(np.linalg.det(l1)) + np.trace(l1.T @ Sigma1 @ l1))
print(- 2 * np.log(np.linalg.det(l2)) + np.trace(l2.T @ Sigma2 @ l2))



7.1588830833596715
7.1588830833596715
